In [1]:
import torch
import src.pre_processing as pp
import src.ResNet18 as rn
import src.engine as eng

In [2]:
# get the data and split 
X, Y, Z, proba = pp.get_data()
X_train, Y_train, Z_train, X_val, Y_val, Z_val, X_test, Y_test, Z_test = pp.split_data(X, Y, Z)

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {device}")

param_grid = {
    'lr': [1e-3, 5e-4, 1e-4, 5e-5, 1e-5],   # Added intermediate steps and lower bound
    'batch_size': [16, 32, 64, 128],        # Added larger batch size
    'weight_decay': [1e-3, 1e-4, 1e-5, 0],  # Added stronger regularization
    'epochs': [3] 
}

best_params = eng.tune_hyperparameters(rn.ResNet18_Grayscale, X_train, Y_train, X_val, Y_val, param_grid, device, n_trials=5)

Using device: mps
--- Starting Hyperparameter Tuning (5 trials) ---

[Trial 1/5] Params: {'lr': 0.001, 'batch_size': 16, 'weight_decay': 0, 'epochs': 3}
   -> Result: Val F1: 0.6884 (Loss: 0.4764)
   *** New Best Model Found! ***

[Trial 2/5] Params: {'lr': 0.0001, 'batch_size': 16, 'weight_decay': 1e-05, 'epochs': 3}
   -> Result: Val F1: 0.6884 (Loss: 0.4764)
   *** New Best Model Found! ***

[Trial 2/5] Params: {'lr': 0.0001, 'batch_size': 16, 'weight_decay': 1e-05, 'epochs': 3}
   -> Result: Val F1: 0.7491 (Loss: 0.3590)
   *** New Best Model Found! ***

[Trial 3/5] Params: {'lr': 0.001, 'batch_size': 16, 'weight_decay': 1e-05, 'epochs': 3}
   -> Result: Val F1: 0.7491 (Loss: 0.3590)
   *** New Best Model Found! ***

[Trial 3/5] Params: {'lr': 0.001, 'batch_size': 16, 'weight_decay': 1e-05, 'epochs': 3}
   -> Result: Val F1: 0.6842 (Loss: 0.4308)

[Trial 4/5] Params: {'lr': 0.001, 'batch_size': 16, 'weight_decay': 0, 'epochs': 3}
   -> Result: Val F1: 0.6842 (Loss: 0.4308)

[Trial 

In [4]:
BATCH_SIZE = best_params['batch_size']
train_loader = eng.create_loader(X_train, Y_train, BATCH_SIZE)
val_loader = eng.create_loader(X_val, Y_val, BATCH_SIZE)
test_loader = eng.create_loader(X_test, Y_test, BATCH_SIZE)

In [5]:
model = rn.ResNet18_Grayscale().to(device)

final_params = best_params.copy()
final_params['epochs'] = 20
final_params['early_stopping_patience'] = 5

model, history, best_metrics = eng.train_model(model, train_loader, val_loader, final_params, device)

# Save Baseline Model and History
import pandas as pd
import os

os.makedirs('results/models', exist_ok=True)
torch.save(model.state_dict(), 'results/models/model_baseline.pth')
pd.DataFrame(history).to_csv('results/history_baseline.csv', index=False)
print("Baseline model and history saved.")

Training on mps for 20 epochs...
Ep 1: TrLoss 0.4484 | Val F1 0.7124 | ValLoss 0.3768 | LR 0.000100
Ep 1: TrLoss 0.4484 | Val F1 0.7124 | ValLoss 0.3768 | LR 0.000100
Ep 2: TrLoss 0.2975 | Val F1 0.7404 | ValLoss 0.3642 | LR 0.000100
Ep 2: TrLoss 0.2975 | Val F1 0.7404 | ValLoss 0.3642 | LR 0.000100
Ep 3: TrLoss 0.2556 | Val F1 0.7373 | ValLoss 0.4044 | LR 0.000100
Ep 3: TrLoss 0.2556 | Val F1 0.7373 | ValLoss 0.4044 | LR 0.000100
Ep 4: TrLoss 0.1912 | Val F1 0.7372 | ValLoss 0.3914 | LR 0.000100
Ep 4: TrLoss 0.1912 | Val F1 0.7372 | ValLoss 0.3914 | LR 0.000100
Ep 5: TrLoss 0.1429 | Val F1 0.7008 | ValLoss 0.5018 | LR 0.000100
Ep 5: TrLoss 0.1429 | Val F1 0.7008 | ValLoss 0.5018 | LR 0.000100


KeyboardInterrupt: 

# Experimenting with the Augmented dataset

In [ ]:
# --- Experiment with Geometric Augmentation ---
import numpy as np

# 1. Create Geometrically Augmented Dataset
# We augment the minority class to balance the dataset
X_train_aug, Y_train_aug, Z_train_aug = pp.augment_data(X_train, Y_train, Z_train)

# Combine original and augmented data
X_train_geo = np.concatenate((X_train, X_train_aug), axis=0)
Y_train_geo = np.concatenate((Y_train, Y_train_aug), axis=0)
Z_train_geo = np.concatenate((Z_train, Z_train_aug), axis=0)

print(f"Training with Geometric Augmentation. New dataset size: {len(X_train_geo)}")

# 2. Tune Hyperparameters on the augmented dataset
best_params_geo = eng.tune_hyperparameters(rn.ResNet18_Grayscale, X_train_geo, Y_train_geo, X_val, Y_val, param_grid, device, n_trials=5)

# 3. Create Loaders using the new best batch size
BATCH_SIZE_GEO = best_params_geo['batch_size']
train_loader_geo = eng.create_loader(X_train_geo, Y_train_geo, BATCH_SIZE_GEO)
val_loader_geo = eng.create_loader(X_val, Y_val, BATCH_SIZE_GEO)
test_loader_geo = eng.create_loader(X_test, Y_test, BATCH_SIZE_GEO)

# 4. Train Model with best parameters
model_geo = rn.ResNet18_Grayscale().to(device)

final_params_geo = best_params_geo.copy()
final_params_geo['epochs'] = 20
final_params_geo['early_stopping_patience'] = 5

model_geo, history_geo, best_metrics_geo = eng.train_model(model_geo, train_loader_geo, val_loader_geo, final_params_geo, device)

# Save Geometric Model and History
torch.save(model_geo.state_dict(), 'results/models/model_geo.pth')
pd.DataFrame(history_geo).to_csv('results/history_geo.csv', index=False)
print("Geometric model and history saved.")

# Experimenting with VAE Generated Dataset
This section loads a pre-trained VAE model to generate synthetic defect samples and augments the training set with them.


In [ ]:
# --- Experiment with VAE Generated Dataset ---
import src.VaeA as vA
import os
import numpy as np

# 1. Load VAE Model
# TODO: Download the weights file and set the correct path here
weights_path = 'vaeA_model_Beta_ann_weights.pth' 

vae_model = vA.VAE(latent_dim=128, img_channels=1)

# Load weights if available
if os.path.exists(weights_path):
    vae_model.load_state_dict(torch.load(weights_path, map_location=device))
    vae_model.to(device)
    vae_model.eval()
    print("VAE model loaded successfully.")
    
    # 2. Prepare Data for Generation
    # We use the geometrically augmented dataset as the source for the VAE to learn/sample from
    
    X_train_vae = pp.get_defect(X_train_geo, Y_train_geo)
    # Manually filter Z for defected samples as pp.get_defect only returns X
    Z_train_vae = Z_train_geo[Y_train_geo == 1]
    
    X_healthy = X_train_geo[Y_train_geo == 0]

    # 3. Generate Samples
    print("Generating samples with VAE...")
    # Note: Adjust n_samples as needed. Notebook.ipynb used 800.
    X_gen = vA.generate_controlled_samples(vae_model, X_train_vae, Z_train_vae, X_healthy, n_samples=800, mode='spherical', threshold=7.0, device=device)
    Y_gen = np.ones(X_gen.shape[0])
    
    print(f"Generated {len(X_gen)} samples.")

    # 4. Create VAE Augmented Dataset (Original + Generated)
    X_train_vae_aug = np.concatenate((X_train, X_gen), axis=0)
    Y_train_vae_aug = np.concatenate((Y_train, Y_gen), axis=0)
    
    print(f"Training with VAE Augmentation. New dataset size: {len(X_train_vae_aug)}")

    # 5. Tune Hyperparameters on the VAE augmented dataset
    best_params_vae = eng.tune_hyperparameters(rn.ResNet18_Grayscale, X_train_vae_aug, Y_train_vae_aug, X_val, Y_val, param_grid, device, n_trials=5)

    # 6. Create Loaders
    BATCH_SIZE_VAE = best_params_vae['batch_size']
    train_loader_vae = eng.create_loader(X_train_vae_aug, Y_train_vae_aug, BATCH_SIZE_VAE)
    val_loader_vae = eng.create_loader(X_val, Y_val, BATCH_SIZE_VAE)
    test_loader_vae = eng.create_loader(X_test, Y_test, BATCH_SIZE_VAE)

    # 7. Train Model
    model_vae = rn.ResNet18_Grayscale().to(device)
    
    final_params_vae = best_params_vae.copy()
    final_params_vae['epochs'] = 20
    final_params_vae['early_stopping_patience'] = 5

    model_vae, history_vae, best_metrics_vae = eng.train_model(model_vae, train_loader_vae, val_loader_vae, final_params_vae, device)

    # Save VAE Model and History
    torch.save(model_vae.state_dict(), 'results/models/model_vae.pth')
    pd.DataFrame(history_vae).to_csv('results/history_vae.csv', index=False)
    print("VAE model and history saved.")

else:
    print(f"WARNING: Weights file not found at {weights_path}. Please download the weights and update the path to proceed with generation and training.")

In [ ]:
# --- Comparative Analysis ---
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, f1_score, precision_score, recall_score
import os
import torch
import numpy as np

# 1. Load Histories
# Assuming files exist if previous cells ran successfully
history_baseline = pd.read_csv('results/history_baseline.csv')
history_geo = pd.read_csv('results/history_geo.csv')

histories = {
    'Baseline': history_baseline,
    'Geometric': history_geo
}

# Check if VAE history exists (since it's conditional)
if os.path.exists('results/history_vae.csv'):
    history_vae = pd.read_csv('results/history_vae.csv')
    histories['VAE'] = history_vae

# 2. Plot Comparative Curves
if histories:
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    # Training Loss
    for name, hist in histories.items():
        axes[0].plot(hist['train_loss'], label=name)
    axes[0].set_title('Training Loss')
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Loss')
    axes[0].legend()
    axes[0].grid(True)
    
    # Validation Loss
    for name, hist in histories.items():
        axes[1].plot(hist['val_loss'], label=name)
    axes[1].set_title('Validation Loss')
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('Loss')
    axes[1].legend()
    axes[1].grid(True)
    
    # Validation F1 Score
    for name, hist in histories.items():
        axes[2].plot(hist['val_f1'], label=name)
    axes[2].set_title('Validation F1 Score')
    axes[2].set_xlabel('Epoch')
    axes[2].set_ylabel('F1 Score')
    axes[2].legend()
    axes[2].grid(True)
    
    plt.tight_layout()
    plt.show()

# 3. Evaluate Models on Test Set
def evaluate_model(model, loader, device):
    model.eval()
    all_preds = []
    all_targets = []
    
    with torch.no_grad():
        for inputs, labels in loader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            preds = torch.sigmoid(outputs) > 0.5
            all_preds.extend(preds.cpu().numpy())
            all_targets.extend(labels.numpy())
            
    return np.array(all_targets), np.array(all_preds)

models_to_eval = {}
if 'model' in locals(): models_to_eval['Baseline'] = model
if 'model_geo' in locals(): models_to_eval['Geometric'] = model_geo
if 'model_vae' in locals(): models_to_eval['VAE'] = model_vae

# If models are not in locals, try loading them
if not models_to_eval:
    print("Models not found in memory. Attempting to load from disk...")
    if os.path.exists('results/models/model_baseline.pth'):
        m = rn.ResNet18_Grayscale().to(device)
        m.load_state_dict(torch.load('results/models/model_baseline.pth', map_location=device))
        models_to_eval['Baseline'] = m
    
    if os.path.exists('results/models/model_geo.pth'):
        m = rn.ResNet18_Grayscale().to(device)
        m.load_state_dict(torch.load('results/models/model_geo.pth', map_location=device))
        models_to_eval['Geometric'] = m
        
    if os.path.exists('results/models/model_vae.pth'):
        m = rn.ResNet18_Grayscale().to(device)
        m.load_state_dict(torch.load('results/models/model_vae.pth', map_location=device))
        models_to_eval['VAE'] = m

# 4. Comparative Table and Confusion Matrices
results_table = []

if models_to_eval:
    fig, axes = plt.subplots(1, len(models_to_eval), figsize=(5 * len(models_to_eval), 4))
    if len(models_to_eval) == 1: axes = [axes]
    
    for i, (name, model_obj) in enumerate(models_to_eval.items()):
        targets, preds = evaluate_model(model_obj, test_loader, device)
        
        # Metrics
        acc = accuracy_score(targets, preds)
        f1 = f1_score(targets, preds)
        prec = precision_score(targets, preds)
        rec = recall_score(targets, preds)
        
        results_table.append({
            'Model': name,
            'Accuracy': acc,
            'F1 Score': f1,
            'Precision': prec,
            'Recall': rec
        })
        
        # Confusion Matrix
        cm = confusion_matrix(targets, preds)
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[i])
        axes[i].set_title(f'{name} Confusion Matrix')
        axes[i].set_xlabel('Predicted')
        axes[i].set_ylabel('Actual')

    plt.tight_layout()
    plt.show()
    
    # Display Table
    results_df = pd.DataFrame(results_table)
    print("\nComparative Results:")
    print(results_df)
else:
    print("No models available for evaluation.")